## Analysis
Cell 1

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import IntegerType
df = spark.table("workspace.us_accidents_project.silver_us_accidents")

print(f"Total Accidents Analyzed: {df.count():,}")
print("Date Range:")
df.select(F.min("Start_Time"), F.max("Start_Time")).show()

df.printSchema()

* We have 1,250,000 accidents in total.
* We have data from 2017-02-04 to 2021-02-28.
* there are 43 different fetures we can explore

The schema is robust, containing a mix of temporal (timestamps), spatial (coordinates), and environmental (weather conditions) features. The data types appear correct (e.g., timestamps are actual TimestampType, not strings), indicating good initial ingestion (Bronze to Silver transformation).

cell 2: Data Completeness (Null Analysis)

In [0]:
from pyspark.sql import functions as F

df = spark.table("workspace.us_accidents_project.silver_us_accidents")
null_counts = df.select([
    (F.count(F.col(c)) / F.count("*")).alias(c) 
    for c in df.columns
])
display(null_counts)

Analysis: The data quality is exceptionally high. Most core columns (ID, Severity, Start_Lat, City) have nearly 100% completeness (1.0).

Weakest Points: Weather-related columns (Wind_Direction, Weather_Condition) show slight drops in completeness (+-97%), which is expected as weather sensors may occasionally fail or be unavailable for specific coordinates.

Conclusion: There are no "trash columns" (columns with a small % of data, e.g. <5% data) evident in the output provided. 

cell 3: Numerical Distribution & Outliers

In [0]:
numerical_cols = ["Distance_km", "accident_duration_hours", "Temperature_C", "Wind_Speed_kmh", "Precipitation_mm"]
stats_df = df.select(numerical_cols).describe()

display(stats_df)

* Critical Anomaly Detected: accident_duration_hours has a maximum value of 26,304 hours (approx. 3 years). This is statistically impossible for a single traffic accident and severely skews the mean (11.2 hours). 

* Distance: The maximum traffic jam/accident length is 2.3 km, which is okay.

* Weather: Temperature_C range (-12°C to 43°C) and Wind_Speed (up to ~29 km/h) look reasonable and within physical limits.


cell 4: temporal integrity

In [0]:
negative_time = df.filter(F.col("accident_duration_hours") < 0)
future_accidents = df.filter(F.year("Start_Time") > 2021)
instant_accidents = df.filter(F.col("accident_duration_hours") == 0)

print(f"Negative Duration Errors: {negative_time.count()}")
print(f"Future Date Errors: {future_accidents.count()}")
print(f"Instant (0s) Accidents: {instant_accidents.count()}")

if negative_time.count() > 0:
    display(negative_time.select("ID", "Start_Time", "End_Time", "accident_duration_hours").limit(10))

The result shows 0 errors for all three logical checks.
No accidents have negative duration.
No accidents are dated in the future (beyond 2021).
No accidents have 0 duration.
Conclusion: The data cleansing process regarding timestamps is functioning correctly; the temporal integrity of the dataset is solid.

cell 5: ghost coordinates

In [0]:
# check the US boundaries
# Lat: 24 - 49 (Mainland), Lng: -125 - -66
# we also check for Alaska and Hawaii

geo_errors = df.filter(
    (F.col("Start_Lat") == 0) | (F.col("Start_Lng") == 0) | 
    (F.col("Start_Lat") > 72) | (F.col("Start_Lat") < 18) | 
    (F.col("Start_Lng") > -60) | (F.col("Start_Lng") < -172)
)
print(f"Geospatial Coordinates Errors: {geo_errors.count()}")
display(geo_errors.select("Start_Lat", "Start_Lng", "City", "State"))

All the points are in the boundaries of the USA

Cell 6: weather anomalies

In [0]:
weather_anomalies = (df
    .groupBy("Weather_Condition")
    .count()
    .orderBy("count") 
)

#check the most rare weather anomalies
display(weather_anomalies.limit(20))

Data Quality: The "rare" values found (e.g., "Funnel Cloud", "Squalls", "Sand") appear to be valid meteorological terms rather than data entry typos (like "Snyy" or "Rian").
It means the data is cleaned enough. 

cell 7: Timezone Bias

In [0]:
timezone_skew = df.groupBy("Timezone").count().orderBy(F.desc("count"))

print("Source Bias:")
display(source_skew)

print("Timezone Representation:")
display(timezone_skew)

Databricks visualization. Run in Databricks to view.

Timezone Distribution: There is a clear population bias.
* US/Pacific (483k) and US/Eastern (474k) dominate the dataset, which correlates with US population centers.
* US/Mountain (85k) is significantly lower, which is expected due to lower population density in that region, rather than missing data.
* Nulls: Only ~2000 records have null timezones (< 0.16%), which is acceptable.

cell 8: logic consistency (teleportation check)

In [0]:
teleport_errors = df.filter(
    (F.col("Start_Lat") == F.col("End_Lat")) & 
    (F.col("Start_Lng") == F.col("End_Lng")) & 
    (F.col("Distance_km") > 0.1) 
)

print(f"Teleportation Artifacts (Moved 0 lat/lng, but Distance > 0): {teleport_errors.count()}")

display(teleport_errors.select("ID", "Start_Lat", "End_Lat", "Distance_km").limit(20))

There are no records where the car moved 0 coordinates but registered a distance traveled. The Distance_km field is geometrically consistent with the lat/long coordinates.

## Interesting things after the general analysis

####lazy state index

In [0]:
lazy_states = (df
    .groupBy("State")
    .agg(
        F.count("ID").alias("Total_Accidents"),
        F.avg("accident_duration_hours").alias("Avg_Clearance_Time"),
        F.max("accident_duration_hours").alias("Max_Clearance_Time")
    )
    .filter(F.col("Total_Accidents") > 5000) 
    .orderBy(F.desc("Avg_Clearance_Time"))
)

display(lazy_states)

Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

In [0]:
from pyspark.sql import functions as F

lazy_states_no_la = (df
    .groupBy("State")
    .agg(
        F.count("ID").alias("Total_Accidents"),
        F.avg("accident_duration_hours").alias("Avg_Clearance_Time"),
        F.max("accident_duration_hours").alias("Max_Clearance_Time")
    )
    .filter(F.col("Total_Accidents") > 5000) 
    .filter(F.col("State") != "LA") 

    .orderBy(F.desc("Avg_Clearance_Time"))
)

display(lazy_states_no_la)

Databricks visualization. Run in Databricks to view.

Cell 1: Operational Inefficiency (State Clearance Times)
Analysis:

The "Louisiana" Anomaly: The result for LA (Louisiana) is statistically impossible for a real-world metric. An average clearance time of 478 hours (nearly 20 days) with a max of 1.3 years indicates that ticket "close" times are not being recorded correctly in that state.

Real vs. Dirty Data: States like NY, FL, TX show averages of 2.9 – 4.3 hours. This is the "true" operational baseline for clearing accidents.

Conclusion: This chart does not actually show "lazy" states; it shows inconsistent data reporting standards. LA, OR, and IN likely leave tickets "open" in the database long after the road is cleared.

cell 3 geospatial artifacts (cursed coordinates)

In [0]:
cursed_spots = (df
    .withColumn("Lat_Round", F.round("Start_Lat", 3))
    .withColumn("Lng_Round", F.round("Start_Lng", 3))
    .groupBy("Lat_Round", "Lng_Round", "City", "State")
    .agg(
        F.count("ID").alias("Crash_Count"),
        F.avg("Severity").alias("Avg_Severity")
    )
    .orderBy(F.desc("Crash_Count"))
    .limit(20)
)

display(cursed_spots)

Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

Hotspot Identification: it identifies valid high-density crash zones.
The Findings:

1) Downey, CA: 320 crashes in a 100m radius.
2) Diamond Bar, CA: 281 crashes.
3) Miami, FL: 223 crashes.
Conclusion: These are not "glitches."

 These are specific major interchanges (e.g., the I-5/I-605 interchange in Downey) or notoriously dangerous intersections. The "rounding to 3 decimals" technique worked perfectly to cluster these events.

 if you look at the map, you can notice that the most dangerous spots are **interchanges or crossroads**

## the I-5/I-605 interchange in Downey

![Downey Crash Hotspot](https://www.planetizen.com/files/styles/featured_large/public/images/Downey.jpg.webp?itok=I8SMiHAN)

In [0]:
cursed_spots_full = (df
    .withColumn("Lat_Round", F.round("Start_Lat", 3))
    .withColumn("Lng_Round", F.round("Start_Lng", 3))
    .groupBy("Lat_Round", "Lng_Round", "City", "State") 
    .agg(
        F.count("ID").alias("Crash_Count"),
        F.avg("Severity").alias("Avg_Severity")
    )
)
pdf_heat = cursed_spots_full.toPandas()

In [0]:
display(cursed_spots_full)

In [0]:
import plotly.express as px
fig = px.density_mapbox(
    pdf_heat, 
    lat='Lat_Round', 
    lon='Lng_Round', 
    z='Crash_Count', 
    radius=10,      
    center=dict(lat=37.0902, lon=-95.7129), 
    zoom=3,
    mapbox_style="carto-darkmatter"
)

fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

On the map you can see the heat map of car accidents

Cell 4. Phantom Jam
Goal: Find High Severity accidents (meaning road closed) that recorded 0 km of traffic impact. This indicates missing data or sensor failure.



In [0]:

phantom_jams = (df
    .filter((F.col("Severity") == 4) & (F.col("Distance_km") == 0))
    .groupBy("State")
    .count()
    .withColumnRenamed("count", "Phantom_Data_Count")
    .orderBy(F.desc("Phantom_Data_Count"))
)
display(phantom_jams)

Databricks visualization. Run in Databricks to view.

Cell 4: "Phantom" Jams (Data Consistency)
Analysis:

The Logical Conflict: A Severity 4 accident is defined as having a significant impact on traffic (road closures). Having 0.0 km of impact distance is a contradiction.

California (1225), Texas (645), and New York (599) have the highest count of these phantom jams.

Conclusion: This represents Missing Data. The sensors detected the severity (likely via duration or keywords) but failed to measure the queue length. These records might need to be filtered out if you are training a machine learning model to predict traffic impact.


Cell 5.1: The "Matrix of Danger" (Temporal Heatmap)

In [0]:

heatmap_data = (df
    .groupBy("start_weekday", "start_hour")
    .agg(F.count("ID").alias("Accident_Volume"))
    .orderBy("start_weekday", "start_hour")
)

display(heatmap_data)

Databricks visualization. Run in Databricks to view.

The "Double Peak" Pattern (Weekdays): Monday through Thursday shows two distinct spikes:

Morning Rush: 7:00 - 8:00 AM (~10k accidents).

Evening Rush: 4:00 - 5:00 PM (~15k accidents).

Insight: The evening rush hour is 50% more dangerous than the morning rush hour. (Likely due to driver fatigue + sun glare + more casual drivers mixing with commuters).

The Weekend Shift: Saturday and Sunday lack the morning spike entirely. They have a gentle "bell curve" that peaks in the mid-afternoon (1:00 PM - 4:00 PM) with much lower overall volume (~5k-6k).

The Deadliest Time: Friday at 16:00 (4 PM) is the single most dangerous hour in the dataset (15,888 accidents).

Cell 5.2: The "Deadly Hours" (Severity vs. Volume)
Goal: Compare Count of accidents vs. Severity by hour. (Expectation: 5 PM has high count, but 3 AM has high severity).

In [0]:
hourly_risk = (df
    .groupBy("start_hour")
    .agg(
        F.count("ID").alias("Volume"),
        F.avg("Severity").alias("Danger_Level")
    )
    .orderBy("start_hour")
)

display(hourly_risk)

Databricks visualization. Run in Databricks to view.

Volume Peak: The classic 5:00 PM (17:00) rush hour sees the highest volume of crashes (87,565), but the severity is average (2.20).

Severity Peak: The most dangerous time to be in a crash is actually 5:00 AM (Severity 2.27).

Conclusion:
5 PM: High congestion = lots of minor "fender benders."
5 AM: Empty roads = high speeds. If you crash at 5 AM, it is likely a high-impact collision.

Cell 6: The Infrastructure Paradox. Does safety infrastructure actually help?
Comparing severity at Traffic Signals vs Open Road

In [0]:
infra_analysis = (df
    .groupBy("Traffic_Signal")
    .agg(
        F.avg("Severity").alias("Avg_Severity"),
        F.avg("accident_duration_hours").alias("Avg_Duration"),
        F.count("ID").alias("Total_Crashes")
    )
)
display(infra_analysis)

Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

The Findings: Accidents at traffic signals are a little bit less severe (2.14 vs 2.22) and are cleared 3x faster (4.7 hours vs. 12.1 hours) than those on the open road.

The "Open Road" Danger: 87% of all recorded accidents happen away from traffic signals.

Conclusion: While intersections feel chaotic, the "stop-and-go" nature limits impact speed. Open roads (likely highways) allow for higher speeds, leading to higher severity and much longer clearance times (likely due to towing logistics on highways).

Cell 7: The "Complacency" Theory (Weather Analysis)

Goal: Prove that clear weather has more accidents than bad weather due to driver complacency.

In [0]:
weather_impact = (df
    .groupBy("is_bad_weather")
    .agg(
        F.count("ID").alias("Total_Accidents"),
        F.avg("Severity").alias("Avg_Severity")
    )
    .withColumn("Condition", F.when(F.col("is_bad_weather"), "Bad Weather").otherwise("Clear/Mild"))
)
display(weather_impact)

Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

Analysis:

Volume Paradox: The vast majority of accidents (~1.1 million or ~90%) occur in Clear/Mild weather. Drivers are lulled into a false sense of security when the sun is shining.

Severity Reality: While rarer, "Bad Weather" accidents are indeed statistically more severe (2.25) than clear weather ones (2.21).

Conclusion: Bad weather reduces the count of drivers (or makes them drive slower/cautiously), but when an accident does happen, the physics of wet/frozen roads leads to slightly higher damage.

Cell 8: The COVID-19 Impact (Yearly Trend)
Goal: See the dip in 2020 and the subsequent rise.

In [0]:
yearly_trend = (df
    .withColumn("Year", F.year("Start_Time"))
    .groupBy("Year")
    .agg(
        F.count("ID").alias("Total_Accidents"),
        F.avg("accident_duration_hours").alias("Avg_Duration")
    )
    .orderBy("Year")
)

display(yearly_trend)

Databricks visualization. Run in Databricks to view.

Contradiction Alert: it expects a "COVID dip," but the data shows a massive 300% explosion in accident records in 2020 (708k vs 262k in 2019).


Critical Insight: This does not mean people crashed more during lockdown. It indicates a change in data collection. The system likely added new data sources (feeds) in 2020, resulting in much higher capture rates but seemingly messier data (longer duration/open tickets). 2021 data is incomplete (cuts off in Feb).

## General Conclusion & Strategic Insights ##
### 1. Data Integrity & The "Silver" Illusion ###
1) While the dataset appears technically clean (robust schema, correct data types, <5% nulls in critical columns), our analysis revealed significant "Silent Data Quality" issues:

2) The 2020 Structural Break: The massive 300% spike in accident volume and average duration in 2020 is not a biological or behavioral trend (e.g., COVID-19). It is a methodological artifact caused by the ingestion of new, high-volume/high-noise data feeds. Comparing 2019 volumes directly to 2020 will lead to false conclusions.

3) The "Louisiana" Anomaly: The state of Louisiana is not "lazy" (478-hour clearance average); rather, their reporting system fails to close tickets, leaving them "active" mathematically for years.

4) Useless Features: The Source column contains zero information gain (100% "Source1") and should be dropped.

### 2. The Nature of Severity
1) Our correlation analysis confirms that Severity is a proxy for Traffic Impact, not necessarily physical damage.

2) Primary Driver: Severity has the strongest positive correlation with Distance_km (traffic queue length).

3) Weak Predictors: Weather factors (Wind, Temp) have negligible correlation with Severity.

4) Phantom Jams: We identified thousands of "Severity 4" (Critical) accidents with 0.0 km impact, primarily in CA, TX, and NY, indicating sensor failures or missing geospatial data.

### 3. Behavioral & Temporal Risk Patterns ###
1) Drivers are most vulnerable when they feel safest:

2) The "Complacency Trap": ~90% of accidents occur in clear/mild weather. Drivers lower their guard when conditions are good.

3) The "Volume vs. Danger" Paradox:

5:00 PM (Rush Hour): Highest probability of crashing (Volume), but likely a minor fender-bender (Low Severity).

5:00 AM (Empty Roads): Lowest probability of crashing, but if you do, it is likely catastrophic (High Severity) due to high speeds on open roads.

4) The "Friday Factor": Friday afternoon (4 PM - 5 PM) is the single most dangerous window of the week, combining accumulated weekly fatigue with high traffic volume.

### 4. Infrastructure Insights ###
1) Intersections vs. Highways: Traffic signals act as safety buffers. Accidents at signals are less severe and cleared 3x faster than accidents on the open road.

2) Geospatial Clustering: The "rounding to 3 decimal places" technique successfully identified legitimate high-risk interchanges (e.g., Downey, CA; Miami, FL), validating the spatial accuracy of the dataset.

### Recommendations for Next Steps ###
1) Data Segmentation: Split the analysis into Pre-2020 (stable historical trends) and Post-2020 (high-volume real-time feeds). Do not aggregate them.

2) Feature Engineering: Create a flag is_high_speed_crash based on the Hour (Late Night) and Traffic_Signal (False) to better predict high-severity events.

3) Cleaning: Filter out records where accident_duration_hours > 24 for general analysis to remove the "Louisiana skew," unless specifically analyzing administrative backlog.

Predictive Modeling: Do not use Source, Temperature, or Wind_Chill for severity prediction. Focus on Start_Hour, Side, Traffic_Signal, and Distance_km.

## Recommendations for advanced analysis (will be improved)

Here are the three most high-value projects to build with this data, ranked by feasibility and business value.

Project 1: The "911 Dispatch Priority" System (Classification)
Goal: Predict the Severity (1-4) of an accident immediately when it is reported, before police arrive.

Why? To help emergency services decide whether to send a tow truck (Severity 2) or an ambulance and fire truck (Severity 4).

The ML Task: Multi-class Classification.

Critical Engineering Step (Based on your EDA):

Avoid Data Leakage: You MUST DROP Distance_km and End_Time from your training features. These are outcomes of the accident. At the moment the accident happens (t=0), you don't know how long the traffic jam will be.

Feature Engineering: You must create an Is_Rush_Hour feature (since you proved 5 PM is high volume) and an Is_Intersection feature (since you proved signals reduce severity).

Imbalance Handling: Your data is heavily skewed toward Severity 2. You will need to use Class Weights or Undersampling (downsample Severity 2) so the model learns to spot the rare Severity 4 cases.

Project 2: The "Waze/Google Maps" ETA Corrector (Regression)
Goal: Predict accident_duration_hours.

Why? To update navigation ETAs. If a crash happens on I-95, will it clear in 30 minutes or 4 hours?

The ML Task: Regression (predicting a continuous number).

Critical Engineering Step:

Outlier Removal: You cannot train on the raw data because of the "Louisiana/Active Ticket" issue. You must filter training data to only include closed accidents (e.g., duration < 24 hours).

Model Choice: Gradient Boosted Trees (XGBoost/LightGBM) usually perform best here because they handle non-linear relationships between "Weather" and "Time of Day" well.

Project 3: "Black Spot" Identification (Unsupervised Learning)
Goal: Identify dangerous road segments that need engineering fixes.

The ML Task: Clustering (DBSCAN or K-Means).

Critical Engineering Step:

Use the coordinate rounding technique you discovered (3 decimal places) to group points.

Run DBSCAN to find clusters of accidents that have High Severity but Low Weather Impact. These are roads that are inherently dangerous regardless of rain/snow.

Recommended Next Step: Build the "Severity Classifier"
This is the most standard ML workflow. I have written a PySpark ML Pipeline file for you. This code prepares the data, handles the categorical variables (like State and Weather_Condition), and trains a Random Forest model.

What this code does:

Cleans: Filters out the 2020 data drift (optional, but recommended).

Featurizes: Converts strings (City, Wind_Direction) into numbers machine learning can understand using StringIndexer and OneHotEncoder.

Protects: Explicitly drops "Cheating" columns (Distance_km, End_Time) to prevent data leakage.
Trains: Uses a Random Forest Classifier.
How to use this code
Copy-paste the code block into a new cell in your Databricks notebook.
Run it. It will take a few minutes because StringIndexers need to scan the whole dataset.
Analyze the Output:
Look at the Confusion Matrix output.

Warning Sign: If you see that the model predicts "Severity 2" for everything, your model is suffering from the class imbalance we discussed. The "ML Engineer" fix for that is to use train_data.sampleBy(...) to downsample the majority class before training.